In [1]:
import os 
import datetime
import calendar
import pandas as pd
import numpy as np
#import seaborn as sns
import matplotlib
matplotlib.use('Agg')

In [2]:
os.chdir('C:/Users/aecat/Dropbox/Docencia/PUC/Magíster Ciencia de Datos/Datos')

## Carga de información


In [3]:
laliga2122 = pd.read_excel('LaLiga21 22.xlsx', )
laliga2021 = pd.read_excel('LaLiga20 21.xlsx', )
laliga1920 = pd.read_excel('LaLiga19 20.xlsx', )
laliga1819 = pd.read_excel('LaLiga18 19.xlsx', )
laliga1718 = pd.read_excel('LaLiga17 18.xlsx', )
laliga1617 = pd.read_excel('LaLiga16 17.xlsx', )
seg2122 =  pd.read_excel('LaLiga123 21 22.xlsx', )
seg2021 =  pd.read_excel('LaLiga123 20 21.xlsx', )
seg1920 =  pd.read_excel('LaLiga123 19 20.xlsx', )
seg1819 =  pd.read_excel('LaLiga123 18 19.xlsx', )
seg1718 =  pd.read_excel('LaLiga123 17 18.xlsx', )
seg1617 =  pd.read_excel('LaLiga123 16 17.xlsx', )

Lista de Dataframes con la información cargada recientemente.

In [4]:
laliga2122.head()

,Wk,Day,Date,Time,Local,xG,Score,xG.1,Visitante,Attendance,Venue,Referee,Informe del partido,Notes
0,1.0,Vie,2021-08-13,21:00 (15:00),Valencia,1.3,1–0,1.2,Getafe,9.868,Estadio de Mestalla,Jesús Gil,Informe del partido,NaN
1,1.0,Sáb,2021-08-14,19:30 (13:30),Cádiz,0.4,1–1,1.0,Levante,6.941,Estadio Nuevo Mirandilla,Santiago Jaime,Informe del partido,NaN
2,1.0,Sáb,2021-08-14,19:30 (13:30),Mallorca,0.4,1–1,0.9,Betis,6.279,Iberostar Estadi,Antonio Matéu Lahoz,Informe del partido,NaN
3,1.0,Sáb,2021-08-14,22:00 (16:00),Osasuna,0.8,0–0,0.7,Espanyol,6.715,Estadio El Sadar,Miguel Ángel Ortiz Arias,Informe del partido,NaN
4,1.0,Sáb,2021-08-14,22:00 (16:00),Alavés,1.7,1–4,1.8,Real Madrid,3.968,Estadio de Mendizorroza,César Soto,Informe del partido,NaN


In [5]:
#creamos un par de listas con la información de los Dataframes que creamos antes
laliga_torneos = [laliga2122,laliga2021,laliga1920,laliga1819,laliga1718,laliga1617]
seg_torneos = [seg2122,seg2021,seg1920,seg1819,seg1718,seg1617]

Se crea un solo Dataframe para los 6 torneos de LaLiga (1era división) y otro Dataframe para los 6 torneos de LaLiga123 (2da división).

In [6]:
laliga = pd.concat([laliga2122,laliga2021,laliga1920,laliga1819,laliga1718,laliga1617], sort = False)

In [7]:
segunda = pd.concat([seg2122,seg2021,seg1920,seg1819,seg1718,seg1617], sort = False)

Queremos obtener una lista con los equipos que hayan estado en primera y en segunda división al menos alguna de las temporadas que tenemos registradas.
Para esto, se busca un match entre los equipos que hayan hecho de local entre ambas divisiones, lo que nos permitirá reconocer a los equipos que han estado en ambas divisiones.

In [8]:
#el comando merge nos permite unir los Dataframes laliga y segunda. O sea, se buscan equipos que hayan jugado de local en primera
#y registramos el partido como Local y Visitante X (en la fecha Date_x) y en segunda como Local y Visitante_y (en la fecha Date_y)
match_1 = laliga.merge(segunda, on = 'Local').loc[:,['Date_x','Local','Visitante_x','Visitante_y','Date_y']]

In [9]:
match_1.head(10)

,Date_x,Local,Visitante_x,Visitante_y,Date_y
0,2021-08-14,Cádiz,Levante,Ponferradina,2019-08-18
1,2021-08-14,Cádiz,Levante,Extremadura,2019-09-01
2,2021-08-14,Cádiz,Levante,Girona,2019-09-14
3,2021-08-14,Cádiz,Levante,La Coruña,2019-09-21
4,2021-08-14,Cádiz,Levante,Huesca,2019-10-02
5,2021-08-14,Cádiz,Levante,Las Palmas,2019-10-18
6,2021-08-14,Cádiz,Levante,Sporting Gijón,2019-11-01
7,2021-08-14,Cádiz,Levante,Lugo,2019-11-24
8,2021-08-14,Cádiz,Levante,Elche,2019-12-08
9,2021-08-14,Cádiz,Levante,Numancia,2019-12-21


Se extraen los equipos equipos que hicieron de local en 1era y 2da división

In [10]:
#del Dataframe anterior nos quedamos solamente con los registros no repetidos, como lista.
equipos_primseg = match_1['Local'].unique().tolist()
equipos_primseg

['Cádiz',
 'Mallorca',
 'Osasuna',
 'Elche',
 nan,
 'Granada',
 'Espanyol',
 'Levante',
 'Getafe',
 'Rayo Vallecano',
 'Eibar',
 'Valladolid',
 'Huesca',
 'Leganés',
 'Girona',
 'La Coruña',
 'Málaga',
 'Las Palmas',
 'Sporting Gijón']

Se crea un diccionario con la key del nombre del equipo y de value una tupla con la cantidad de veces que estuvo en primera y la cantidad de veces que estuvo en segunda.

In [11]:
temp = {} #creamos el diccionario temp
for equipo in equipos_primseg: #recorremos la lista recién creado de equipos que estuvieron en primera y segunda
    cont_p = 0 #contador para cantidad de torneos en primera
    cont_s = 0 #contador para cantidad de torneos en segunda
    for torneo in laliga_torneos: #recorremos la lista con los partidos de primera
        if equipo in torneo['Local'].unique().tolist():
            cont_p+=1
    for tor in seg_torneos:
        if equipo in tor['Local'].unique().tolist():
            cont_s+=1
    #incorporamos en el diccionario el resultado del conteo de la cantidad de torneos 
    #en que ese equipo participó en primera y en segunda. La "key" del diccionario será el nombre del equipo
    temp[equipo] = (cont_p,cont_s)   

In [12]:
temp

{'Cádiz': (2, 4),
 'Mallorca': (2, 3),
 'Osasuna': (4, 2),
 'Elche': (2, 3),
 nan: (6, 6),
 'Granada': (4, 2),
 'Espanyol': (5, 1),
 'Levante': (5, 1),
 'Getafe': (5, 1),
 'Rayo Vallecano': (2, 4),
 'Eibar': (5, 1),
 'Valladolid': (3, 3),
 'Huesca': (2, 4),
 'Leganés': (4, 2),
 'Girona': (2, 4),
 'La Coruña': (2, 2),
 'Málaga': (2, 4),
 'Las Palmas': (2, 4),
 'Sporting Gijón': (1, 5)}

## Visualización de los resultados obtenidos

In [13]:
#transformamos el diccionario "temp" en el Dataframe "df"
df = pd.DataFrame(temp)
df1 = df.transpose()
#le colocamos los nombres que deseamos a las columnas de este Dataframe "df"
df1.reset_index().set_axis(['Equipos','Temporadas primera división','Temporadas segunda división'],axis = 'columns')

,Equipos,Temporadas primera división,Temporadas segunda división
0,Cádiz,2,4
1,Mallorca,2,3
2,Osasuna,4,2
3,Elche,2,3
4,NaN,6,6
5,Granada,4,2
6,Espanyol,5,1
7,Levante,5,1
8,Getafe,5,1
9,Rayo Vallecano,2,4


## Ejercicio 1
¿Cómo es el código que permite construir un único Dataframe con todos los partidos jugados por el “Real Madrid”? 

In [14]:
#Para esto, buscamos en el Dataframe "laliga" (que tiene todos los partidos de primera división de las 6 temporadas) los partidos
#en que el Real Madrid ha sido local o ha sido visita
realmadrid = laliga[(laliga['Local']=='Real Madrid')|(laliga['Visitante']=='Real Madrid')]
realmadrid

,Wk,Day,Date,Time,Local,xG,Score,xG.1,Visitante,Attendance,Venue,Referee,Informe del partido,Notes
4,1.0,Sáb,2021-08-14,22:00 (16:00),Alavés,1.7,1–4,1.8,Real Madrid,3.968,Estadio de Mendizorroza,César Soto,Informe del partido,NaN
18,2.0,Dom,2021-08-22,22:00 (16:00),Levante,1.9,3–3,1.2,Real Madrid,9.838,Estadio Ciudad de Valencia,Guillermo Cuadra,Informe del partido,NaN
27,3.0,Sáb,2021-08-28,22:00 (16:00),Betis,0.8,0–1,1.1,Real Madrid,22.590,Estadio Benito Villamarín,Alejandro Hernández,Informe del partido,NaN
38,4.0,Dom,2021-09-12,21:00 (16:00),Real Madrid,3.4,5–2,1.8,Celta Vigo,19.874,Estadio Santiago Bernabéu,José Sánchez,Informe del partido,NaN
50,5.0,Dom,2021-09-19,21:00 (16:00),Valencia,0.8,1–2,1.1,Real Madrid,26.689,Estadio de Mestalla,Pablo González,Informe del partido,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,35.0,Sáb,2017-04-29,16:15 (11:15),Real Madrid,NaN,2–1,NaN,Valencia,75.180,Estadio Santiago Bernabéu,Jesús Gil,Informe del partido,NaN
390,36.0,Sáb,2017-05-06,20:45 (15:45),Granada,NaN,0–4,NaN,Real Madrid,19.094,Estadio Nuevo Los Cármenes,Ignacio Iglesias,Informe del partido,NaN
405,37.0,Dom,2017-05-14,20:00 (14:00),Real Madrid,NaN,4–1,NaN,Sevilla,79.356,Estadio Santiago Bernabéu,Alberto Undiano,Informe del partido,NaN
408,21.0,Mié,2017-05-17,21:00 (15:00),Celta Vigo,NaN,1–4,NaN,Real Madrid,22.838,Estadio de Balaídos,Juan Martínez,Informe del partido,NaN
